## student scrapper activity
scrape categories and all books for each category and then insert into mongo db

In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pymongo
from webdriver_manager.chrome import ChromeDriverManager
import pdb
from selenium.common.exceptions import NoSuchElementException,ElementClickInterceptedException,TimeoutException
import time
import pandas as pd
import numpy as np

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240


[WDM] - Driver [C:\Users\arken\.wdm\drivers\chromedriver\win32\86.0.4240.22\chromedriver.exe] found in cache


In [3]:
def author_data():
        browser.find_by_text('(about)')[i].click()
        html = browser.html
        soup_about = BeautifulSoup(html,'html.parser')
        date_born = soup_about.find_all('span', class_='author-born-date')[0].text
        location = soup_about.find_all('span', class_='author-born-location')[0].text
        born = f'{date_born} {location}'
        description = soup_about.find_all('div', class_='author-description')[0].text
        browser.back()
        return(born,description)

In [4]:
url = 'http://quotes.toscrape.com/'
#url = 'http://quotes.toscrape.com/page/10/'
browser.visit(url)
quotes_list = []

#if browser.find_by_text('Next ').first:
i=0
for x in range(10):
    # HTML object
    html = browser.html
    # Parse HTML with Beautiful Soup
    soup = BeautifulSoup(html,'html.parser')
    quotes = soup.find_all('div', class_='quote')
    for quote in quotes:
        tag_list = []
        tags = quote.find_all('a', class_='tag')
        for tag in tags:
            tag_list.append(tag.text)
        
        born,description = author_data()

        #pdb.set_trace()
        data = {'quote' : quote.span.text,
                'author' : quote.small.text,
                'tags' : tag_list,
                'born': born,
                'description': description
               }
        quotes_list.append(data)
    i=i+1
    time.sleep(0.25)
    try:    
        print('Clicking!')
        browser.find_by_text('Next ').first.click()
        time.sleep(0.25)
    except:
        print('Done Scraping')

quotes_list

Clicking!
Clicking!
Clicking!
Clicking!
Clicking!
Clicking!
Clicking!
Clicking!
Clicking!
Clicking!
Done Scraping


[{'quote': '“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”',
  'author': 'Albert Einstein',
  'tags': ['change', 'deep-thoughts', 'thinking', 'world'],
  'born': 'March 14, 1879 in Ulm, Germany',
  'description': '\n        In 1879, Albert Einstein was born in Ulm, Germany. He completed his Ph.D. at the University of Zurich by 1909. His 1905 paper explaining the photoelectric effect, the basis of electronics, earned him the Nobel Prize in 1921. His first paper on Special Relativity Theory, also published in 1905, changed the world. After the rise of the Nazi party, Einstein made Princeton his permanent home, becoming a U.S. citizen in 1940. Einstein, a pacifist during World War I, stayed a firm proponent of social justice and responsibility. He chaired the Emergency Committee of Atomic Scientists, which organized to alert the public to the dangers of atomic warfare.At a symposium, he advised: "In their struggle for th

In [5]:
type(quotes_list)

list

In [6]:
browser.quit()

In [7]:
pd.DataFrame(quotes_list)

,quote,author,tags,born,description
0,“The world as we have created it is a process ...,Albert Einstein,"[change, deep-thoughts, thinking, world]","March 14, 1879 in Ulm, Germany","\n In 1879, Albert Einstein was born in..."
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,"[abilities, choices]","March 14, 1879 in Ulm, Germany","\n In 1879, Albert Einstein was born in..."
2,“There are only two ways to live your life. On...,Albert Einstein,"[inspirational, life, live, miracle, miracles]","March 14, 1879 in Ulm, Germany","\n In 1879, Albert Einstein was born in..."
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,"[aliteracy, books, classic, humor]","March 14, 1879 in Ulm, Germany","\n In 1879, Albert Einstein was born in..."
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,"[be-yourself, inspirational]","March 14, 1879 in Ulm, Germany","\n In 1879, Albert Einstein was born in..."
...,...,...,...,...,...
95,“You never really understand a person until yo...,Harper Lee,[better-life-empathy],"September 20, 1948 in Bayonne, New Jersey, The...",\n George R. R. Martin was born Septemb...
96,“You have to write the book that wants to be w...,Madeleine L'Engle,"[books, children, difficult, grown-ups, write,...","September 20, 1948 in Bayonne, New Jersey, The...",\n George R. R. Martin was born Septemb...
97,“Never tell the truth to people who are not wo...,Mark Twain,[truth],"September 20, 1948 in Bayonne, New Jersey, The...",\n George R. R. Martin was born Septemb...
98,"“A person's a person, no matter how small.”",Dr. Seuss,[inspirational],"September 20, 1948 in Bayonne, New Jersey, The...",\n George R. R. Martin was born Septemb...


In [8]:
import pymongo
import datetime

In [9]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [10]:
db=client.quotes

In [11]:
quotes = db.quotes

In [12]:
# quotes.collection.bulkWrite([{insert_many}:{quotes_list}])

In [13]:
quotes.insert_many(quotes_list)